# The Hundred Prisoner Problem
## Problem Statement
The hundred prisoner problem is a mathematical problem that seeks the optimal solution to the following riddle: 100 prisoners are indexed by numbers which are placed it to 100 envelopes, not necessarily (and indeed, not typically!) in the same order. Each prisoner independently goes in and selects 50 envelopes. If all prisoners find their own number, then all prisoners are freed.

## Solutions
### Random Selection
The naive solution to this problem would involve random choice on the part of each prisoner. But this has an exceptionally low probability of success. Since each prisoner has a 50% chance of success, we would expect the probability that all succeed is (0.5)^100. Virtually no chance at freedom!

### The Loop Method
A better solution exists by reframing the problem in terms of closed loops. Consider this: each prisoner chooses the envelope corresponding to their index, then the envelope corresponding to the index in the previous envelope. The last envelope they choose, if successful, would lead them back to the envelope they first chose (if the process didn't terminate). Thus this is a problem of finding closed loops! Furthermore, the probability of success is the  probability that there is no loop greater in size than 50. Since longer loops are rarer in a random system, this is far more likely to succeed than the random method. 

A corollary is that if prisoners are allowed to swap two envelopes, they can always succeed, as they can divide a loop of size 100 in the worst case into two loops of size 50!

It has been shown that the probability of success with the loop method asymptotically approaches 30% as the number of prisoners increases, and that it is the optimal solution for the problem.

## The Experiment!
When I heard of this problem, I immediately became interested in the base cases. The loop method is indeed optimal as the number of prisoners gets larger, but is there ever a time it *isn't* optimal at small values? And how does the probability change at small values?

For example, for 2 prisoners, we'd expect about a 25% chance of success with the random selection, as each prisoner has a 50% chance of success (0.5 * 0.5 = 0.25). For the loop selection, we'd actually expect a 50% chance of success overall. A single selection is equivalent to the random chance selection at 2 prisoners, but the truth value of it guarantees the truth value of the other selection, since the prisoners must have different numbers (so they are either both right or both wrong).

What about with 4? Furthermore, what if we extend this to odd numbers? We'd have to figure out whether to raise or lower the number of attempts, since odd numbers are not divisible by 2, but how would this impact the probability of success? Would it be a significant change?

That's where I started. Along the way, I ended up with even more questions which we'll tackle later here. For now, let's simulate the basic prisoner problem.

## I. The Basic Hundred-Prisoner Problem

In [1]:
from runner import Simulation
from algorithms import RandomSelect, LoopSelect

In [2]:
config = {'nPrisoners': 100,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': True,
              'chunks': 10,
              'nEnvelopes': 100,
              }

In [3]:
sim = Simulation(config)

In [4]:
sim.simulate()

Progress: Currently on trial 1000. This chunk took 18.331271409988403 seconds to complete. Estimating 164.98140835762024 more seconds.
Progress: Currently on trial 2000. This chunk took 19.429975748062134 seconds to complete. Estimating 151.04480743408203 more seconds.
Progress: Currently on trial 3000. This chunk took 20.977933645248413 seconds to complete. Estimating 139.50475919246674 more seconds.
Progress: Currently on trial 4000. This chunk took 22.33263611793518 seconds to complete. Estimating 126.7855657339096 more seconds.
Progress: Currently on trial 5000. This chunk took 23.415758848190308 seconds to complete. Estimating 111.36664465069771 more seconds.
Progress: Currently on trial 6000. This chunk took 24.735366582870483 seconds to complete. Estimating 94.01733237504959 more seconds.
Progress: Currently on trial 7000. This chunk took 25.91818881034851 seconds to complete. Estimating 74.13373636454344 more seconds.
Progress: Currently on trial 8000. This chunk took 27.176493

In [5]:
# We can view our data as a pandas dataframe. The rows are prisoners, the columns are trials. For example:
sim.data['LoopSelect']

,Trial 0,Trial 1,Trial 2,Trial 3,Trial 4,Trial 5,Trial 6,Trial 7,Trial 8,Trial 9,...,Trial 9990,Trial 9991,Trial 9992,Trial 9993,Trial 9994,Trial 9995,Trial 9996,Trial 9997,Trial 9998,Trial 9999
0,False,False,True,False,True,False,True,True,False,True,...,False,True,True,True,False,True,True,True,True,False
1,False,True,True,False,True,False,True,True,False,True,...,False,True,False,True,True,False,False,True,True,True
2,False,False,True,False,True,False,True,True,False,True,...,False,True,False,True,False,True,False,True,True,True
3,False,True,True,True,True,False,True,True,False,True,...,False,True,False,False,False,False,True,True,True,False
4,False,False,True,False,True,True,True,True,False,True,...,False,True,True,True,False,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,True,True,False,True,False,True,True,False,True,...,False,True,True,False,False,False,True,True,True,False
96,False,False,True,False,True,True,True,True,False,True,...,False,True,True,True,False,False,False,True,True,False
97,False,False,True,False,True,False,True,True,False,True,...,True,True,False,True,False,False,True,True,True,False
98,False,False,True,False,True,True,True,True,False,True,...,True,True,False,True,False,False,False,True,True,False


In [6]:
# There's a built-in stats function that tells us basic info, such as the number of successful trials (trials where every prisoner found their envelope and is marked True).
sim.stats()['success']

{'RandomSelect': 0, 'LoopSelect': 3087}

Okay, so for 100 trials the results roughly agree with our expectations: RandomSelect rarely succeeds, LoopSelect succeeds about half the time! Furhter, we can double-check that the LoopSelect successes correspond to those trials where the loop size was less than 50. First, naively:

In [7]:
(sim.extra['LoopSelect']['loop_size'] < 50).sum().value_counts()

100    2880
50      206
47      200
49      196
37      187
41      184
34      180
46      177
48      177
40      176
38      175
45      175
44      168
42      168
43      166
39      157
32      156
23      151
36      148
27      144
31      144
33      142
28      142
18      140
21      139
30      139
7       138
29      136
20      134
24      130
17      127
35      127
22      126
25      121
14      118
19      116
2       115
26      114
16      114
10      113
15      110
0       109
4       108
5       108
8       107
13      106
12      105
11      103
1       102
9       100
3        99
6        97
dtype: int64

This tells us that 29 of our trials had a loop size less than 29 for all prisoners, which corresponds to the amount of successes indeed! The other trials had some subset of prisoners who had a loop size less than 50, but other prisoners were stuck in a bigger loop, so they couldn't succeed and the whole trial failed. Let's check the correspondence more directly.

In [8]:
import numpy as np
loop_lt_50 = np.where((sim.extra['LoopSelect']['loop_size'] <= 50).sum() == 100)[0]
loop_lt_50

array([   2,    4,    6, ..., 9991, 9997, 9998])

In [9]:
successful_trials = np.where(sim.data['LoopSelect'].sum() == 100)[0]
successful_trials

array([   2,    4,    6, ..., 9991, 9997, 9998])

In [10]:
loop_lt_50 == successful_trials

array([ True,  True,  True, ...,  True,  True,  True])

We have thus proven that successful trials are those with a loop of size less than 50! Everything seems to be working out. Let's dive into the questions I posed earlier.

## II. Base Cases
What are the results with only small numbers of prisoners? By our prediction in the introduction, we'd expect about a 25% chance of success if the prisoners choose randomly and a 50% chance of success if they use the loop method (as they will both either be correct or both be wrong).

Let's run a few trials to see if that prediction is correct.

In [11]:
bc2_config = {'nPrisoners': 2,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': True,
              'chunks': 10,
              }

In [12]:
bc2_sim = Simulation(bc2_config)

In [13]:
bc2_sim.simulate()

Progress: Currently on trial 1000. This chunk took 2.1743791103363037 seconds to complete. Estimating 19.569381952285767 more seconds.
Progress: Currently on trial 2000. This chunk took 2.2735955715179443 seconds to complete. Estimating 17.791767120361328 more seconds.
Progress: Currently on trial 3000. This chunk took 2.478100299835205 seconds to complete. Estimating 16.457149863243103 more seconds.
Progress: Currently on trial 4000. This chunk took 2.5711491107940674 seconds to complete. Estimating 14.766420722007751 more seconds.
Progress: Currently on trial 5000. This chunk took 2.6222994327545166 seconds to complete. Estimating 12.70835354924202 more seconds.
Progress: Currently on trial 6000. This chunk took 2.7975594997406006 seconds to complete. Estimating 10.678381264209747 more seconds.
Progress: Currently on trial 7000. This chunk took 2.938586473464966 seconds to complete. Estimating 8.412227623164654 more seconds.
Progress: Currently on trial 8000. This chunk took 3.072743

**(Note for further optimization: for larger amounts of trials, the time for chunks to run seems to increase, though it should be roughly constant since a chunk represents a uniform amount of trials. This might be due to the way we're adding to our dataframe. A more numpythonic approach, getting rid of the Trial class and treating Trials as an array, would likely perform better. I may troubleshoot this further, but for my own personal purposes this is sufficient for now. A progress bar would also be a nice addition!)**

This looks promising!

In [14]:
bc2_sim.stats()['success']

{'RandomSelect': 2536, 'LoopSelect': 4981}

Now we can try the base case of 4, instead.

In [15]:
bc4_config = {'nPrisoners': 4,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': True,
              'chunks': 10,
              }

bc4_sim = Simulation(bc4_config)

bc4_sim.simulate()

Progress: Currently on trial 1000. This chunk took 2.205854892730713 seconds to complete. Estimating 19.852659702301025 more seconds.
Progress: Currently on trial 2000. This chunk took 2.35707950592041 seconds to complete. Estimating 18.251619338989258 more seconds.
Progress: Currently on trial 3000. This chunk took 2.4816572666168213 seconds to complete. Estimating 16.670787930488586 more seconds.
Progress: Currently on trial 4000. This chunk took 2.5992350578308105 seconds to complete. Estimating 14.942248463630676 more seconds.
Progress: Currently on trial 5000. This chunk took 2.7182815074920654 seconds to complete. Estimating 13.02157148718834 more seconds.
Progress: Currently on trial 6000. This chunk took 2.8767266273498535 seconds to complete. Estimating 10.962017953395844 more seconds.
Progress: Currently on trial 7000. This chunk took 3.061664342880249 seconds to complete. Estimating 8.70321211963892 more seconds.
Progress: Currently on trial 8000. This chunk took 3.243118762

In [16]:
bc4_sim.stats()['success']

{'RandomSelect': 573, 'LoopSelect': 4175}

Doing the math for random selection, we'd expect 0.5^4 = 0.0625 success rate, or 62.5 successful trials.

In [17]:
bc3_lower_config = {'nPrisoners': 3,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': True,
              'chunks': 10,
              }

bc3_lower_sim = Simulation(bc3_lower_config)

bc3_lower_sim.simulate()

Progress: Currently on trial 1000. This chunk took 2.1912894248962402 seconds to complete. Estimating 19.72156834602356 more seconds.
Progress: Currently on trial 2000. This chunk took 2.311372756958008 seconds to complete. Estimating 18.01052951812744 more seconds.
Progress: Currently on trial 3000. This chunk took 2.4966564178466797 seconds to complete. Estimating 16.617808997631073 more seconds.
Progress: Currently on trial 4000. This chunk took 2.6089394092559814 seconds to complete. Estimating 14.948658406734467 more seconds.
Progress: Currently on trial 5000. This chunk took 2.733456611633301 seconds to complete. Estimating 13.062177672982216 more seconds.
Progress: Currently on trial 6000. This chunk took 2.8177762031555176 seconds to complete. Estimating 10.860362440347672 more seconds.
Progress: Currently on trial 7000. This chunk took 2.9825422763824463 seconds to complete. Estimating 8.54640569910407 more seconds.
Progress: Currently on trial 8000. This chunk took 3.11165380

In [18]:
bc3_raise_config = {'nPrisoners': 3,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': False,
              'chunks': 10,
              }

bc3_raise_sim = Simulation(bc3_raise_config)

bc3_raise_sim.simulate()

Progress: Currently on trial 1000. This chunk took 2.1820006370544434 seconds to complete. Estimating 19.637979984283447 more seconds.
Progress: Currently on trial 2000. This chunk took 2.348879337310791 seconds to complete. Estimating 18.12339210510254 more seconds.
Progress: Currently on trial 3000. This chunk took 2.4527995586395264 seconds to complete. Estimating 16.513691544532776 more seconds.
Progress: Currently on trial 4000. This chunk took 2.6151275634765625 seconds to complete. Estimating 14.922600388526917 more seconds.
Progress: Currently on trial 5000. This chunk took 2.787257671356201 seconds to complete. Estimating 13.185801953077316 more seconds.
Progress: Currently on trial 6000. This chunk took 2.879847764968872 seconds to complete. Estimating 11.033964335918427 more seconds.
Progress: Currently on trial 7000. This chunk took 3.042714834213257 seconds to complete. Estimating 8.70176524668932 more seconds.
Progress: Currently on trial 8000. This chunk took 3.140657424

In [19]:
print('lower:', bc3_lower_sim.stats()['success'])
print('raise:', bc3_raise_sim.stats()['success'])

lower: {'RandomSelect': 372, 'LoopSelect': 1620}
raise: {'RandomSelect': 2693, 'LoopSelect': 6665}


How about the odd cases? The stricter condition on our experiment is that we lower the number of attempts to the nearest integer. For 3 prisoners, they each get 1 attempt. We could also loosen the problem by giving them an extra attempt, so 3 prisoners get 2. How much does this impact the results? As far as the random algorithm works, lowering in this case should give us (33%)^3 = 3.6% and raising should give (66%)^3 = 28.7%. A wide discrepency! Of course, we'd expect the discrepency to decease with higher numbers of prisoners. Let's see!

In [20]:
bc100_raise_config = {'nPrisoners': 9,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': False,
              'chunks': 10,
              }

bc100_raise_sim = Simulation(bc100_raise_config)

bc100_raise_sim.simulate()

Progress: Currently on trial 1000. This chunk took 2.5214054584503174 seconds to complete. Estimating 22.69261908531189 more seconds.
Progress: Currently on trial 2000. This chunk took 2.6316001415252686 seconds to complete. Estimating 20.61190414428711 more seconds.
Progress: Currently on trial 3000. This chunk took 2.7036633491516113 seconds to complete. Estimating 18.480398774147034 more seconds.
Progress: Currently on trial 4000. This chunk took 3.172229528427124 seconds to complete. Estimating 17.436776518821716 more seconds.
Progress: Currently on trial 5000. This chunk took 3.7783329486846924 seconds to complete. Estimating 16.71107605099678 more seconds.
Progress: Currently on trial 6000. This chunk took 3.2370927333831787 seconds to complete. Estimating 13.158558189868927 more seconds.
Progress: Currently on trial 7000. This chunk took 3.3218212127685547 seconds to complete. Estimating 9.917150013148785 more seconds.
Progress: Currently on trial 8000. This chunk took 3.4300408

In [21]:
bc100_lower_config = {'nPrisoners': 9,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': True,
              'chunks': 10,
              }

bc100_lower_sim = Simulation(bc100_raise_config)

bc100_lower_sim.simulate()

Progress: Currently on trial 1000. This chunk took 2.3891842365264893 seconds to complete. Estimating 21.5026216506958 more seconds.
Progress: Currently on trial 2000. This chunk took 2.5354928970336914 seconds to complete. Estimating 19.698589324951172 more seconds.
Progress: Currently on trial 3000. This chunk took 2.645392417907715 seconds to complete. Estimating 17.87691032886505 more seconds.
Progress: Currently on trial 4000. This chunk took 2.8466482162475586 seconds to complete. Estimating 16.201393246650696 more seconds.
Progress: Currently on trial 5000. This chunk took 3.012799024581909 seconds to complete. Estimating 14.282510727643967 more seconds.
Progress: Currently on trial 6000. This chunk took 3.222728967666626 seconds to complete. Estimating 12.158406436443329 more seconds.
Progress: Currently on trial 7000. This chunk took 3.351308584213257 seconds to complete. Estimating 9.586322374641895 more seconds.
Progress: Currently on trial 8000. This chunk took 3.5237317085

In [22]:
print('lower:', bc100_lower_sim.stats()['success'])
print('raise:', bc100_raise_sim.stats()['success'])

lower: {'RandomSelect': 36, 'LoopSelect': 4502}
raise: {'RandomSelect': 40, 'LoopSelect': 4537}


So it has indeed closed up significantly! Note that going above 10 prisoners will get virtually no successses in the random selection (unless you go to an extremely high amount of trials).

## III. Disentangling Envelopes

Part of the constraints of the prisoner problem is that the number of envelopes is equal to the number of prisoners. What if it wasn't? That is, what if we could have 200 envelopes for 100 prisoners? The envelopes would still have the first hundred indices of the prisoners, but there would be extra indices that correspond to no prisoner. Nominally, for the (even) random selection algorithm, this should drop each success chance to 25%. What about for the loop selection?

In [23]:
de24_config = {'nPrisoners': 2,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': True,
              'chunks': 10,
              'nEnvelopes': 4
              }

de24_sim = Simulation(de24_config)

de24_sim.simulate()

Progress: Currently on trial 1000. This chunk took 2.1988470554351807 seconds to complete. Estimating 19.789591312408447 more seconds.
Progress: Currently on trial 2000. This chunk took 2.3513565063476562 seconds to complete. Estimating 18.200695991516113 more seconds.
Progress: Currently on trial 3000. This chunk took 2.5014443397521973 seconds to complete. Estimating 16.717760384082794 more seconds.
Progress: Currently on trial 4000. This chunk took 2.6957709789276123 seconds to complete. Estimating 15.251987278461456 more seconds.
Progress: Currently on trial 5000. This chunk took 2.7574923038482666 seconds to complete. Estimating 13.248659893870354 more seconds.
Progress: Currently on trial 6000. This chunk took 2.8976688385009766 seconds to complete. Estimating 11.094748228788376 more seconds.
Progress: Currently on trial 7000. This chunk took 3.0132999420166016 seconds to complete. Estimating 8.680440086871386 more seconds.
Progress: Currently on trial 8000. This chunk took 3.162

In [24]:
de24_sim.stats()['success']

{'RandomSelect': 2381, 'LoopSelect': 4122}

Recall the earlier result.

In [25]:
bc2_sim.stats()['success']

{'RandomSelect': 2536, 'LoopSelect': 4981}

The discrepency in the number of envelopes seems to lower the probability of success for the loop selection. Interesting! What if we crank up the number of envelopes?

In [26]:
de232_config = {'nPrisoners': 2,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': True,
              'chunks': 10,
              'nEnvelopes': 32
              }

de232_sim = Simulation(de232_config)

de232_sim.simulate()

Progress: Currently on trial 1000. This chunk took 2.263608455657959 seconds to complete. Estimating 20.372435331344604 more seconds.
Progress: Currently on trial 2000. This chunk took 2.4138259887695312 seconds to complete. Estimating 18.709616661071777 more seconds.
Progress: Currently on trial 3000. This chunk took 2.5639305114746094 seconds to complete. Estimating 17.15911477804184 more seconds.
Progress: Currently on trial 4000. This chunk took 2.680448293685913 seconds to complete. Estimating 15.395173251628876 more seconds.
Progress: Currently on trial 5000. This chunk took 2.813387393951416 seconds to complete. Estimating 13.448057845234871 more seconds.
Progress: Currently on trial 6000. This chunk took 2.9643454551696777 seconds to complete. Estimating 11.307857781648636 more seconds.
Progress: Currently on trial 7000. This chunk took 3.151966094970703 seconds to complete. Estimating 8.968345385044813 more seconds.
Progress: Currently on trial 8000. This chunk took 3.20407891

In [27]:
de232_sim.stats()['success']

{'RandomSelect': 2355, 'LoopSelect': 3855}

Can we make random select more successful?

In [28]:
de28192_config = {'nPrisoners': 2,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': True,
              'chunks': 10,
              'nEnvelopes': 8192
              }

de28192_sim = Simulation(de28192_config)

de28192_sim.simulate()

Progress: Currently on trial 1000. This chunk took 40.386916399002075 seconds to complete. Estimating 363.4822111129761 more seconds.
Progress: Currently on trial 2000. This chunk took 41.3705849647522 seconds to complete. Estimating 327.02986907958984 more seconds.
Progress: Currently on trial 3000. This chunk took 40.91786575317383 seconds to complete. Estimating 286.2880027294159 more seconds.
Progress: Currently on trial 4000. This chunk took 41.49070119857788 seconds to complete. Estimating 247.1668782234192 more seconds.
Progress: Currently on trial 5000. This chunk took 41.998693227767944 seconds to complete. Estimating 207.98286139965057 more seconds.
Progress: Currently on trial 6000. This chunk took 41.562302112579346 seconds to complete. Estimating 166.31769347190857 more seconds.
Progress: Currently on trial 7000. This chunk took 42.175464391708374 seconds to complete. Estimating 125.63228836655617 more seconds.
Progress: Currently on trial 8000. This chunk took 42.56852746

**(Note for further optimization: more envelopes does not seem to impede performance, so this part works fine!)**

In [29]:
de28192_sim.stats()['success']

{'RandomSelect': 2362, 'LoopSelect': 3721}

So the loop selection seems to converge asymptotically as envelopes increase, and it's always better! What about for the base case of 3, with both raising and lowering?

In [30]:
de36561_lower_config = {'nPrisoners': 3,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': True,
              'chunks': 10,
              'nEnvelopes': 6561
              }

de36561_lower_sim = Simulation(de36561_lower_config)

de36561_lower_sim.simulate()

Progress: Currently on trial 1000. This chunk took 44.781084299087524 seconds to complete. Estimating 403.02972435951233 more seconds.
Progress: Currently on trial 2000. This chunk took 45.26901459693909 seconds to complete. Estimating 360.20025634765625 more seconds.
Progress: Currently on trial 3000. This chunk took 45.8918354511261 seconds to complete. Estimating 318.20893609523773 more seconds.
Progress: Currently on trial 4000. This chunk took 45.50319027900696 seconds to complete. Estimating 272.88474118709564 more seconds.
Progress: Currently on trial 5000. This chunk took 46.066744565963745 seconds to complete. Estimating 228.86870458722115 more seconds.
Progress: Currently on trial 6000. This chunk took 45.93828105926514 seconds to complete. Estimating 183.42398768663406 more seconds.
Progress: Currently on trial 7000. This chunk took 46.0289032459259 seconds to complete. Estimating 137.82730769366026 more seconds.
Progress: Currently on trial 8000. This chunk took 47.15021347

In [31]:
de36561_raise_config = {'nPrisoners': 3,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': False,
              'chunks': 10,
              'nEnvelopes': 6561
              }

de36561_raise_sim = Simulation(de36561_raise_config)

de36561_raise_sim.simulate()

Progress: Currently on trial 1000. This chunk took 46.050514221191406 seconds to complete. Estimating 414.4546022415161 more seconds.
Progress: Currently on trial 2000. This chunk took 45.79687738418579 seconds to complete. Estimating 367.3894453048706 more seconds.
Progress: Currently on trial 3000. This chunk took 45.43332886695862 seconds to complete. Estimating 319.7494348883629 more seconds.
Progress: Currently on trial 4000. This chunk took 46.05675220489502 seconds to complete. Estimating 275.20564502477646 more seconds.
Progress: Currently on trial 5000. This chunk took 46.53906440734863 seconds to complete. Estimating 231.01660646498203 more seconds.
Progress: Currently on trial 6000. This chunk took 46.6788854598999 seconds to complete. Estimating 185.76435819268227 more seconds.
Progress: Currently on trial 7000. This chunk took 46.987061738967896 seconds to complete. Estimating 140.14217865094543 more seconds.
Progress: Currently on trial 8000. This chunk took 46.5361392498

In [32]:
print('lower:', de36561_lower_sim.stats()['success'])
print('raise:', de36561_raise_sim.stats()['success'])

lower: {'RandomSelect': 1129, 'LoopSelect': 3317}
raise: {'RandomSelect': 1209, 'LoopSelect': 3336}


Still better! It seems the loop selection is a definitively better algorithm, even in the context of the loose odd experiment. Perhaps the one exception is for the case of 1 prisoner with 2 envelopes, in which case we'd expect the probability to be equivalent.

In [33]:
de12_config = {'nPrisoners': 1,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'schema': 'default',
              'lower': False,
              'chunks': 10,
              'nEnvelopes': 2
              }

de12_sim = Simulation(de12_config)

de12_sim.simulate()

Progress: Currently on trial 1000. This chunk took 2.1832973957061768 seconds to complete. Estimating 19.6496422290802 more seconds.
Progress: Currently on trial 2000. This chunk took 2.4022979736328125 seconds to complete. Estimating 18.342251777648926 more seconds.
Progress: Currently on trial 3000. This chunk took 2.4405391216278076 seconds to complete. Estimating 16.566530287265778 more seconds.
Progress: Currently on trial 4000. This chunk took 2.464020013809204 seconds to complete. Estimating 14.491922914981842 more seconds.
Progress: Currently on trial 5000. This chunk took 2.596179246902466 seconds to complete. Estimating 12.528682574629784 more seconds.
Progress: Currently on trial 6000. This chunk took 2.8371520042419434 seconds to complete. Estimating 10.685724586248398 more seconds.
Progress: Currently on trial 7000. This chunk took 2.796790361404419 seconds to complete. Estimating 8.202293638139963 more seconds.
Progress: Currently on trial 8000. This chunk took 2.92309999

In [34]:
de12_sim.stats()['success']

{'RandomSelect': 5103, 'LoopSelect': 4986}

# IV. Different Schemas for Number of Attempts
This section is a work in progress, but if you want to experiment for yourself, the 'modified' schema along with a numerial parameter will give the prisoners a number of attempts equivalent to nEnvelopes/parameter instead of half the envelopes. A parameter of 2 returns default behavior.

In [35]:
mod_config = {'nPrisoners': 100,
              'nTrials': 10000,
              'algorithms': (RandomSelect, LoopSelect),
              'lower': False,
              'chunks': 10,
              'schema': 'modified',
              'param': 2
              }

mod_sim = Simulation(mod_config)

mod_sim.simulate()

Progress: Currently on trial 1000. This chunk took 18.61385464668274 seconds to complete. Estimating 167.52467036247253 more seconds.
Progress: Currently on trial 2000. This chunk took 18.975748538970947 seconds to complete. Estimating 150.35828590393066 more seconds.
Progress: Currently on trial 3000. This chunk took 19.437804460525513 seconds to complete. Estimating 133.81396639347076 more seconds.
Progress: Currently on trial 4000. This chunk took 20.249434232711792 seconds to complete. Estimating 118.09705746173859 more seconds.
Progress: Currently on trial 5000. This chunk took 20.232482433319092 seconds to complete. Estimating 99.7882430255413 more seconds.
Progress: Currently on trial 6000. This chunk took 20.879459857940674 seconds to complete. Estimating 81.67415350675583 more seconds.
Progress: Currently on trial 7000. This chunk took 21.443773984909058 seconds to complete. Estimating 62.79342455416918 more seconds.
Progress: Currently on trial 8000. This chunk took 21.975439

In [36]:
mod_sim.stats()['success'], sim.stats()['success']

({'RandomSelect': 0, 'LoopSelect': 3033},
 {'RandomSelect': 0, 'LoopSelect': 3087})